In [36]:
#author: James Chan © 2018
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import st_utils as ut
import datetime as dt
import nltk
from dateutil import parser
from datetime import timedelta
%matplotlib inline

In [24]:
# Initialize date range
start_date = '2018-01-01'
end_date = '2018-3-31'

In [25]:
# Grab news.  Only need to do this once
keywords = ['tesla'] #these three stocks move largely base on news
news_source = 'wsj.com, bloomberg.com, cnbc.com'
df = ut.get_news(keywords, start_date, end_date, news_source)
df.to_csv('news_dataset.csv')

{'status': 'ok', 'totalResults': 575, 'articles': []}
total number of articles  575
downloading page:  1
downloading page:  2
downloading page:  3
downloading page:  4
downloading page:  5
downloading page:  6


In [26]:
# Get Tesla's prices
dates = pd.date_range(start_date, end_date)
file = 'TSLA.csv'
df = ut.get_data(file, dates)

In [27]:
# Calculate return 
df['Return'] = df['Adj Close'].pct_change()
df.drop(df.index[0], inplace=True)

In [32]:
df_news = pd.read_csv('news_dataset.csv', encoding="ISO-8859-1")

In [35]:
# Clean Dataset
# 1. Change date format to pandas datetime
# 2. Remove any row with title less than 5 words or body less than 10 words.
# 3. Remove any row with entries that have more than 3 question marks.
df_news

,Unnamed: 0,Published,Title,Body,Keyword,Source
0,0,2018-03-28T03:17:00Z,Tesla defends Autopilot record after Feds laun...,Tesla is defending its Autopilot driver-assist...,tesla,CNBC
1,1,2018-03-29T22:12:34Z,Tesla Recalls 123000 Model S Cars Over Bolt Is...,Wall Street Journal Tesla Recalls 123000 Model...,tesla,The Wall Street Journal
2,2,2018-01-26T22:08:00Z,Tesla superfan with stage 4 cancer gets surpri...,Matthew Chans close friend arranged a visit t...,tesla,CNBC
3,3,2018-01-23T19:40:00Z,Elon Musk's Tesla pay proposal could earn him ...,"Tesla CEO Elon Musk earns a token salary, but ...",tesla,CNBC
4,4,2018-01-04T16:05:00Z,Tesla Model 3 production snarls have Wall Stre...,Tesla shares drop after the company's Model 3 ...,tesla,CNBC
5,5,2018-03-27T22:46:00Z,'Tesla is going to come roaring back': Silicon...,Tesla shares dropped more than 8 percent after...,tesla,CNBC
6,6,2018-03-31T03:13:00Z,Tesla says crashed vehicle had been on autopil...,Tesla said on Friday that vehicle logs from la...,tesla,CNBC
7,7,2018-01-23T13:50:08Z,Tesla changes Elon Musks pay to tie it to mar...,The 10-year compensation plan ultimately aims ...,tesla,The Wall Street Journal
8,8,2018-01-03T21:33:00Z,"Tesla delivers 1,550 Model 3 sedans and 29,870...","Tesla delivers 1,550 Model 3 sedans and 29,870...",tesla,CNBC
9,9,2018-03-29T14:17:00Z,Elon Musk Might Control Tesla - Bloomberg,Wall Street Journal Elon Musk Might Control Te...,tesla,Bloomberg


In [50]:
def map_to_close_date(published_date):
    dt = parser.parse(published_date[:-1]) #-1 to ignore the Z, which is GMT.
    dt = dt - timedelta(hours=20) #shift back by 20 hrs in order to map published time to the previous close date. see figure.1
    return dt

In [52]:
df_news['Published'] = df_news['Published'].apply(map_to_close_date)
df_news

,Unnamed: 0,Published,Title,Body,Keyword,Source
0,0,2018-03-27 07:17:00,Tesla defends Autopilot record after Feds laun...,Tesla is defending its Autopilot driver-assist...,tesla,CNBC
1,1,2018-03-29 02:12:34,Tesla Recalls 123000 Model S Cars Over Bolt Is...,Wall Street Journal Tesla Recalls 123000 Model...,tesla,The Wall Street Journal
2,2,2018-01-26 02:08:00,Tesla superfan with stage 4 cancer gets surpri...,Matthew Chans close friend arranged a visit t...,tesla,CNBC
3,3,2018-01-22 23:40:00,Elon Musk's Tesla pay proposal could earn him ...,"Tesla CEO Elon Musk earns a token salary, but ...",tesla,CNBC
4,4,2018-01-03 20:05:00,Tesla Model 3 production snarls have Wall Stre...,Tesla shares drop after the company's Model 3 ...,tesla,CNBC
5,5,2018-03-27 02:46:00,'Tesla is going to come roaring back': Silicon...,Tesla shares dropped more than 8 percent after...,tesla,CNBC
6,6,2018-03-30 07:13:00,Tesla says crashed vehicle had been on autopil...,Tesla said on Friday that vehicle logs from la...,tesla,CNBC
7,7,2018-01-22 17:50:08,Tesla changes Elon Musks pay to tie it to mar...,The 10-year compensation plan ultimately aims ...,tesla,The Wall Street Journal
8,8,2018-01-03 01:33:00,"Tesla delivers 1,550 Model 3 sedans and 29,870...","Tesla delivers 1,550 Model 3 sedans and 29,870...",tesla,CNBC
9,9,2018-03-28 18:17:00,Elon Musk Might Control Tesla - Bloomberg,Wall Street Journal Elon Musk Might Control Te...,tesla,Bloomberg
